# IMDB Movies

## Overview
- For this project, produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tmdbsimple as tmdb

In [4]:
basics_url= 'https://datasets.imdbws.com/title.basics.tsv.gz'

akas_url= 'https://datasets.imdbws.com/title.akas.tsv.gz'

ratings_url= 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [5]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

akas = pd.read_csv(akas_url, sep='\t', low_memory= False)

ratings = pd.read_csv(ratings_url, sep='\t', low_memory= False)

## Cleaning the Data

In [6]:
basics.replace({'\\N':np.nan}, inplace=True)

akas.replace({'\\N':np.nan}, inplace= True)

ratings.replace({'\\N':np.nan}, inplace= True)

### Basics

In [7]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9995235 entries, 0 to 9995234
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 686.3+ MB


In [9]:
basics.dropna(subset = ['startYear'], inplace=True)
basics.dropna(subset = ['runtimeMinutes'], inplace=True)
basics.dropna(subset = ['genres'], inplace=True)

In [10]:
basics = basics[basics.titleType == 'movie']
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [11]:
basics = basics.astype({'startYear': float})

In [12]:
basics = basics[basics.startYear > 1999]

In [13]:
basics = basics[basics.startYear < 2022]

In [14]:
basics['startYear'].unique()

array([2021., 2001., 2020., 2018., 2005., 2002., 2009., 2017., 2000.,
       2006., 2004., 2008., 2007., 2003., 2012., 2010., 2013., 2011.,
       2015., 2016., 2014., 2019.])

In [15]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

### AKAs

In [16]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [17]:
akas = akas[akas.region == 'US']
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [18]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34803       True
61115       True
67667       True
86794       True
93931       True
           ...  
9994908     True
9994917     True
9994956    False
9995001     True
9995085    False
Name: tconst, Length: 138771, dtype: bool

In [19]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86794,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93931,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9994373,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9994768,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9994908,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9994917,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


### Ratings

In [20]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1985
1,tt0000002,5.8,265
2,tt0000003,6.5,1844
3,tt0000004,5.5,178
4,tt0000005,6.2,2626


In [21]:
keepers_rate = ratings['tconst'].isin(akas['titleId'])
keepers_rate

0           True
1           True
2          False
3          False
4           True
           ...  
1327942    False
1327943    False
1327944    False
1327945    False
1327946    False
Name: tconst, Length: 1327947, dtype: bool

In [22]:
ratings = ratings[keepers_rate]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1985
1,tt0000002,5.8,265
4,tt0000005,6.2,2626
5,tt0000006,5.1,182
6,tt0000007,5.4,820
...,...,...,...
1327908,tt9916200,8.1,230
1327909,tt9916204,8.2,264
1327916,tt9916348,8.3,18
1327917,tt9916362,6.4,5411


In [23]:
import os
os.makedirs('Data/',exist_ok=True) 
os.listdir("Data/")

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [24]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [25]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [26]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [27]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [28]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [29]:
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1985
1,tt0000002,5.8,265
2,tt0000005,6.2,2626
3,tt0000006,5.1,182
4,tt0000007,5.4,820


In [31]:
with open('/Users/willrodemoyer/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
    
login.keys()

dict_keys(['API Key'])

In [33]:
tmdb.API_KEY =  login['API Key']